### Code Hist.

 - CODE  
    &ensp; : Crawling - 특일 정보 조회 (KASI)  
    
 - DESC  
    &ensp; : 전처리 - 한국지역난방공사 열판매량/열공급량   
    &emsp; 1) Open API  
    &emsp;&ensp;&ensp;   
    &emsp;&ensp;&ensp;   
    &emsp; 2) 결측치 없음.  

  - DATE  
    &ensp; 2023-07-14 Created  
    &ensp; 2023-11-29 Lab Updated  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1) 직접 다운로드 및 API 사용 가능  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; : https://www.data.go.kr/data/15099319/fileData.do#layer_data_infomation  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2)   
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 3)   

 - DATA  
    &emsp; <"Input">  
    1) None (Input Dataset)  
    &emsp;- Period :   
    &emsp;- Interval : 

    &emsp; <"Output">  
    1) Hourly (관측소/년도별 출력)  
    &nbsp;df_data_cal.to_csv(data_dir + 'KASI_DATE_D_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    2) Daily (관측소/년도별 출력)  
    &nbsp;df_data_cal_24.to_csv(data_dir + 'KASI_DATE_H_Final.csv', index = False, encoding='utf-8-sig')  
    &emsp;- Columns : ['locdate', 'YEAR', 'MONTH', 'DAY'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'dateKind', 'code_day_of_the_week', 'day_of_the_week'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'rest_YN', 'name_of_holiday', 'dist_from_holiday'  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;, 'HOUR', 'MINUTE']
    &emsp;- Period :   
    &emsp;- Interval :  
    
    

 - Related Link  
    &ensp; : 

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [5]:
#region Basic_Import
## Basic
import os
os.path.dirname(os.path.abspath('__file__'))
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import random

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

## 정규화
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
## Modeling, Model Training
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
## Model 평가
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score   # model.score   

## Excel/CSV
import openpyxl, xlrd

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

In [6]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [7]:
## Import_Local (현재 프로그램에서 미사용)
from Src_Dev_Common import DEV_Common_Data_Datetime as com_date
from Src_Dev_Common import DEV_KASI_Holiday as com_Holi
from Src_Dev_Common import DEV_KMA_Weather_ASOS as com_KMA
from Src_Dev_Common import DEV_KDHC_Usage as com_KDHC

### 01-01-02. Config (Directory, Params)

In [8]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [9]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y = pd.datetime.now().year
str_now_m = pd.datetime.now().month
str_now_d = pd.datetime.now().day
str_now_hr = pd.datetime.now().hour
str_now_min = pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-01-08 12:48:31.109058
2024 / 1 / 8
12 : 48


In [10]:
# Define data root directory
srt_dir_kdhcHeat = "../data/data_Energy_KDHC_National_Heat/"
print(os.listdir(srt_dir_kdhcHeat))

['KDHC_COMB_HEAT-CJ_ASOS_KASI_2010-2023.csv', 'KDHC_COMB_HEATSxPCA01_2019-2022_Scaled.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022_complete_analysis.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022_linearReg.csv', 'KDHC_COMB_HEATxWEATHER_2019-2022_mean_imp.csv', 'KDHC_COMB_HEAT_2019-2022.csv', 'KDHC_COMB_KANGNAMHEATxWEATHER_2019-2022.csv', 'KDHC_HEAT', 'KDHC_HEAT.zip', 'KDHC_HEAT_19Region_hr.csv', 'KDHC_HEAT_CHEONGJU_2010_2021.csv', 'KDHC_HEAT_CHEONGJU_2010_2021_1st_Check.csv', 'KDHC_HEAT_SEJONG_2014_2018.csv', 'KDHC_HEAT_SEJONG_2014_2018_1st_Check.csv', 'KDHC_NATIONAL_HEAT_hr.csv', 'KDHC_NATIONAL_HEAT_RAW_hr.csv']


In [11]:
## HEAT_KDHC Parameter
str_ver = "v20220930"

str_key = "xJIsgEs9N%2BvHRawyc%2BE0rnNmiuwcC3OLNugJukgsPmn8v6Zaa1T4Kp8mGO3rJlizFdOzJcrRoW%2Bt%2BYroQAAGRw%3D%3D"

## 01-02. Data Load (df_raw)

In [12]:
Data_KDHC_HEAT = pd.DataFrame()
# List_Page = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']
for page in range(0, 100):
    try:
        Data_KDHC_HEAT_tmp = pd.DataFrame()
        Data_KDHC_HEAT_tmp = com_KDHC.KDHC_HEAT_Usage(str_key, page, str_ver)

        Data_KDHC_HEAT = pd.concat([Data_KDHC_HEAT, pd.DataFrame(Data_KDHC_HEAT_tmp)])

    except Exception as e:
        print(e)
        continue
Data_KDHC_HEAT

,강남,고양,광교,광주전남,김해,단위,대구,동탄,분당,삼송,...,실적일자,양산,요일,용인,중앙,청주,파주,판교,평택,화성
0,61.3,73.0,14.0,3,8,Gcal,19.0,21,55.1,8.0,...,2019-10-01,10,화,30.0,20.0,16,20,8.0,0.5,11.0
1,61.3,65.0,7.0,2,7,Gcal,16.0,16,56.0,12.0,...,2019-10-01,9,화,25.0,18.0,13,15,4.0,0.5,12.0
2,56.3,58.0,11.0,3,7,Gcal,15.0,14,51.0,12.0,...,2019-10-01,8,화,18.0,17.0,10,14,5.0,0.5,12.0
3,53.3,56.0,6.0,2,6,Gcal,13.0,18,48.9,9.0,...,2019-10-01,8,화,17.0,15.0,11,14,6.0,0.5,13.0
4,54.3,48.0,2.0,2,6,Gcal,13.0,15,44.9,11.0,...,2019-10-01,8,화,23.0,14.0,11,14,6.0,0.5,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,120.0,81.0,40.0,6,12,Gcal,30.0,61,74.4,44.0,...,2022-09-30,18,금,87.0,45.0,36,41,37.0,12.0,39.0
326,112.0,90.0,25.0,5,11,Gcal,35.0,55,78.4,46.0,...,2022-09-30,22,금,91.0,46.0,30,40,36.0,4.0,36.0
327,95.0,95.0,16.0,6,12,Gcal,32.0,45,65.4,39.0,...,2022-09-30,22,금,81.0,44.0,27,37,24.0,2.0,29.0
328,89.0,98.0,16.0,6,11,Gcal,34.0,32,51.4,35.0,...,2022-09-30,22,금,80.0,41.0,25,32,19.0,3.0,25.0


In [13]:
Data_KDHC_HEAT = Data_KDHC_HEAT[['실적일자', '요일', '시간'
                                 , '파주', '고양', '삼송' ## 1군
                                 , '중앙' ## 2군
                                 , '강남' ## 3군
                                 , '판교', '분당', '용인', '광교','수원', '화성', '동탄' ## 4군
                                 , '평택', '청주', '세종' ## 5군
                                 , '김해', '대구', '양산' ## 6군
                                 , '광주전남' ## 7군
                                 , '단위']]
Data_KDHC_HEAT.to_csv(srt_dir_kdhcHeat + 'KDHC_NATIONAL_HEAT_RAW_hr.csv')
Data_KDHC_HEAT

,실적일자,요일,시간,파주,고양,삼송,중앙,강남,판교,분당,...,화성,동탄,평택,청주,세종,김해,대구,양산,광주전남,단위
0,2019-10-01,화,1,20,73.0,8.0,20.0,61.3,8.0,55.1,...,11.0,21,0.5,16,18.0,8,19.0,10,3,Gcal
1,2019-10-01,화,2,15,65.0,12.0,18.0,61.3,4.0,56.0,...,12.0,16,0.5,13,15.0,7,16.0,9,2,Gcal
2,2019-10-01,화,3,14,58.0,12.0,17.0,56.3,5.0,51.0,...,12.0,14,0.5,10,15.0,7,15.0,8,3,Gcal
3,2019-10-01,화,4,14,56.0,9.0,15.0,53.3,6.0,48.9,...,13.0,18,0.5,11,16.0,6,13.0,8,2,Gcal
4,2019-10-01,화,5,14,48.0,11.0,14.0,54.3,6.0,44.9,...,13.0,15,0.5,11,18.0,6,13.0,8,2,Gcal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2022-09-30,금,20,41,81.0,44.0,45.0,120.0,37.0,74.4,...,39.0,61,12.0,36,44.0,12,30.0,18,6,Gcal
326,2022-09-30,금,21,40,90.0,46.0,46.0,112.0,36.0,78.4,...,36.0,55,4.0,30,44.0,11,35.0,22,5,Gcal
327,2022-09-30,금,22,37,95.0,39.0,44.0,95.0,24.0,65.4,...,29.0,45,2.0,27,43.0,12,32.0,22,6,Gcal
328,2022-09-30,금,23,32,98.0,35.0,41.0,89.0,19.0,51.4,...,25.0,32,3.0,25,40.0,11,34.0,22,6,Gcal


In [14]:
Data_KDHC_HEAT['실적일자'] = pd.to_datetime(Data_KDHC_HEAT['실적일자'])
Data_KDHC_HEAT['dt_tmp'] = 0
# df_hr  = Data_KDHC_HEAT['시간']
for i in range(0, len(Data_KDHC_HEAT)):
    Data_KDHC_HEAT['dt_tmp'].iloc[i] = str(str(Data_KDHC_HEAT['실적일자'].iloc[i].year)
                                            + str(Data_KDHC_HEAT['실적일자'].iloc[i].month).zfill(2)
                                            + str(Data_KDHC_HEAT['실적일자'].iloc[i].day).zfill(2)
                                            + str(Data_KDHC_HEAT['시간'].iloc[i]).zfill(2))
Data_KDHC_HEAT['dt_tmp']

0      2019100101
1      2019100102
2      2019100103
3      2019100104
4      2019100105
          ...    
325    2022093020
326    2022093021
327    2022093022
328    2022093023
329    2022093024
Name: dt_tmp, Length: 27303, dtype: object

In [15]:
# Data_KDHC_HEAT = conv_midnight_24to00(Data_KDHC_HEAT, 'METER_DATE', 'dt_tmp', '%Y%m%d%H')

# Data_KDHC_HEAT = Data_KDHC_HEAT.rename(columns = {'파주' : "1_PAJU", '고양' : "1_GOYANG", '삼송' : "1_SAMSONG"
#                                                   , '중앙' : "2_JOONGANG"
#                                                   , '강남' : "3_KANGNAM"
#                                                   , '판교' : "4_PANGYO", '분당' : "4_BUNDANG", '용인' : "4_YONGIN", '광교' : "4_GWANGGYO", '수원' : "4_SUWON", '화성' : "4_HWASUNG", '동탄' : "4_DONGTAN"
#                                                   , '평택' : "5_PYONGTAEK", '청주' : "5_CHEONGJU", '세종' : "5_SEJONG"
#                                                   , '김해' : "6_KIMHAE", '대구' : "6_DAEGU", '양산' : "6_YANGSAN"
#                                                   , '광주전남' : "7_GWANGJU_JEONNAM"})
# Data_KDHC_HEAT = Data_KDHC_HEAT[['METER_DATE'
#                                  , '1_PAJU', '1_GOYANG', '1_SAMSONG' ## 1군
#                                  , '2_JOONGANG' ## 2군
#                                  , '3_KANGNAM' ## 3군
#                                  , '4_PANGYO', '4_BUNDANG', '4_YONGIN', '4_GWANGGYO','4_SUWON', '4_HWASUNG', '4_DONGTAN' ## 4군
#                                  , '5_PYONGTAEK', '5_CHEONGJU', '5_SEJONG' ## 5군
#                                  , '6_KIMHAE', '6_DAEGU', '6_YANGSAN' ## 6군
#                                  , '7_GWANGJU_JEONNAM']] ## 7군
# print(Data_KDHC_HEAT.columns)
# Data_KDHC_HEAT.to_csv(srt_dir_kdhcHeat + 'KDHC_NATIONAL_HEAT_hr.csv')
# Data_KDHC_HEAT.info()

In [17]:
Data_KDHC_HEAT = com_date.conv_midnight_24to00(Data_KDHC_HEAT, 'METER_DATE', 'dt_tmp', '%Y%m%d%H')

Data_KDHC_HEAT = Data_KDHC_HEAT.rename(columns = {'파주' : "PAJU", '고양' : "GOYANG", '삼송' : "SAMSONG"
                                                  , '중앙' : "JOONGANG"
                                                  , '강남' : "KANGNAM"
                                                  , '판교' : "PANGYO", '분당' : "BUNDANG", '용인' : "YONGIN", '광교' : "GWANGGYO", '수원' : "SUWON", '화성' : "HWASUNG", '동탄' : "DONGTAN"
                                                  , '평택' : "PYONGTAEK", '청주' : "CHEONGJU", '세종' : "SEJONG"
                                                  , '김해' : "KIMHAE", '대구' : "DAEGU", '양산' : "YANGSAN"
                                                  , '광주전남' : "GWANGJU_JEONNAM"})
Data_KDHC_HEAT = Data_KDHC_HEAT[['METER_DATE'
                                 , 'PAJU', 'GOYANG', 'SAMSONG' ## 1군
                                 , 'JOONGANG' ## 2군
                                 , 'KANGNAM' ## 3군
                                 , 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO','SUWON', 'HWASUNG', 'DONGTAN' ## 4군
                                 , 'PYONGTAEK', 'CHEONGJU', 'SEJONG' ## 5군
                                 , 'KIMHAE', 'DAEGU', 'YANGSAN' ## 6군
                                 , 'GWANGJU_JEONNAM']] ## 7군
print(Data_KDHC_HEAT.columns)
Data_KDHC_HEAT.to_csv(srt_dir_kdhcHeat + 'KDHC_NATIONAL_HEAT_hr.csv')
Data_KDHC_HEAT.info()

Index(['METER_DATE', 'PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG', 'KANGNAM',
       'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON', 'HWASUNG',
       'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG', 'KIMHAE', 'DAEGU',
       'YANGSAN', 'GWANGJU_JEONNAM'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 27303 entries, 0 to 329
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   METER_DATE       27303 non-null  object
 1   PAJU             27303 non-null  int64 
 2   GOYANG           27303 non-null  object
 3   SAMSONG          27303 non-null  object
 4   JOONGANG         27303 non-null  object
 5   KANGNAM          27303 non-null  object
 6   PANGYO           27303 non-null  object
 7   BUNDANG          27303 non-null  object
 8   YONGIN           27303 non-null  object
 9   GWANGGYO         27303 non-null  object
 10  SUWON            27303 non-null  object
 11  HWASUNG          27303 non-n